In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import time

In [32]:
df = pd.read_csv('file2.csv', dtype={'Unnamed': 'float', 'FIRE_YEAR': 'int', 'STAT_CAUSE_DESCR':'str',
                                     'STAT_CAUSE_CODE':'str', 'LATITUDE':'float','LONGITUDE':'float', 'STATE':'str','FIRE_SIZE':'float',
                                     'DISCOVERY_DATE':'float','DISCOVERY_TIME':'float','CONT_DATE':'float','CONT_TIME':'float',
                                     'DISCOVERY_DATE_Greg':'str','CONT_DATE_Greg':'str','DAYS_SPENT':'float','HRS_SPENT':'float'})
df.head()

,Unnamed: 0,FIRE_YEAR,STAT_CAUSE_DESCR,STAT_CAUSE_CODE,LATITUDE,LONGITUDE,STATE,FIRE_SIZE,DISCOVERY_DATE,DISCOVERY_TIME,CONT_DATE,CONT_TIME,DISCOVERY_DATE_Greg,CONT_DATE_Greg,DISCOVERY_MONTH,DISCOVERY_DAY_OF_WEEK,DAYS_SPENT,HRS_SPENT
0,0,2005,Miscellaneous,9.0,40.036944,-121.005833,CA,0.10,2453403.5,1300.0,2453403.5,1730.0,2005-02-02,2005-02-02,2,Wednesday,0.0,4.50
1,1,2004,Lightning,1.0,38.933056,-120.404444,CA,0.25,2453137.5,845.0,2453137.5,1530.0,2004-05-12,2004-05-12,5,Wednesday,0.0,6.75
2,2,2004,Debris Burning,5.0,38.984167,-120.735556,CA,0.10,2453156.5,1921.0,2453156.5,2024.0,2004-05-31,2004-05-31,5,Monday,0.0,1.05
3,3,2004,Lightning,1.0,38.559167,-119.913333,CA,0.10,2453184.5,1600.0,2453189.5,1400.0,2004-06-28,2004-07-03,6,Monday,5.0,118.00
4,4,2004,Lightning,1.0,38.559167,-119.933056,CA,0.10,2453184.5,1600.0,2453189.5,1200.0,2004-06-28,2004-07-03,6,Monday,5.0,116.00


In [33]:
print(" \nCount total NaN at each column in a DataFrame : \n\n",
      df.isnull().sum())

 
Count total NaN at each column in a DataFrame : 

 Unnamed: 0                    0
FIRE_YEAR                     0
STAT_CAUSE_DESCR              0
STAT_CAUSE_CODE               0
LATITUDE                      0
LONGITUDE                     0
STATE                         0
FIRE_SIZE                     0
DISCOVERY_DATE                0
DISCOVERY_TIME           882638
CONT_DATE                891531
CONT_TIME                972553
DISCOVERY_DATE_Greg           0
CONT_DATE_Greg           891531
DISCOVERY_MONTH               0
DISCOVERY_DAY_OF_WEEK         0
DAYS_SPENT               891531
HRS_SPENT                988458
dtype: int64


In [3]:
df = df.drop(columns=['Unnamed: 0','STAT_CAUSE_CODE','DISCOVERY_DATE_Greg','CONT_DATE_Greg','DISCOVERY_DATE','CONT_DATE'])
df = df.dropna()
df.head()

,FIRE_YEAR,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,STATE,FIRE_SIZE,DISCOVERY_TIME,CONT_TIME,DISCOVERY_MONTH,DISCOVERY_DAY_OF_WEEK,DAYS_SPENT,HRS_SPENT
0,2005,Miscellaneous,40.036944,-121.005833,CA,0.10,1300.0,1730.0,2,Wednesday,0.0,4.50
1,2004,Lightning,38.933056,-120.404444,CA,0.25,845.0,1530.0,5,Wednesday,0.0,6.75
2,2004,Debris Burning,38.984167,-120.735556,CA,0.10,1921.0,2024.0,5,Monday,0.0,1.05
3,2004,Lightning,38.559167,-119.913333,CA,0.10,1600.0,1400.0,6,Monday,5.0,118.00
4,2004,Lightning,38.559167,-119.933056,CA,0.10,1600.0,1200.0,6,Monday,5.0,116.00


In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['STAT_CAUSE_DESCR'] = le.fit_transform(df['STAT_CAUSE_DESCR'])
df['STATE'] = le.fit_transform(df['STATE'])
df['DISCOVERY_DAY_OF_WEEK'] = le.fit_transform(df['DISCOVERY_DAY_OF_WEEK'])

In [5]:
y = df['STAT_CAUSE_DESCR']
X = df.drop(columns = ['STAT_CAUSE_DESCR'])
from sklearn.model_selection import cross_val_score, StratifiedKFold
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
print("X_dev: ", X_dev.shape)
print("X_test: ", X_test.shape)
print("y_dev: ", y_dev.shape)
print("y_test: ", y_test.shape)
X_dev.head()
X_test.head()

X_dev:  (713605, 11)
X_test:  (178402, 11)
y_dev:  (713605,)
y_test:  (178402,)


,FIRE_YEAR,LATITUDE,LONGITUDE,STATE,FIRE_SIZE,DISCOVERY_TIME,CONT_TIME,DISCOVERY_MONTH,DISCOVERY_DAY_OF_WEEK,DAYS_SPENT,HRS_SPENT
102718,1997,31.011667,-86.541667,1,0.1,1400.0,1430.0,8,5,0.0,0.500000
1341995,2005,43.031521,-73.593330,34,0.9,1610.0,1610.0,4,5,0.0,0.000000
1729060,2014,48.660300,-98.884800,28,96.4,1200.0,1230.0,11,4,0.0,0.500000
1327640,2004,42.926690,-76.576524,34,0.1,1938.0,1938.0,8,1,0.0,0.000000
305165,2000,65.715400,-150.101400,0,30.0,1830.0,1311.0,6,6,9.0,210.683333


Gradient Boosting Classifier

In [34]:
from sklearn.ensemble import GradientBoostingClassifier
import time

start_time = time.time()
parameters = {"n_estimators": [5, 10],
            "max_depth": [5,10],
            "learning_rate": [0.1, 1]}

GBC = GridSearchCV(GradientBoostingClassifier(), parameters, scoring = 'accuracy', verbose = 5, cv = 2)
GBC.fit(X_dev, y_dev)

print("Time for model selection : {} seconds".format(time.time() - start_time))
print("Best Hyperparameters : ", GBC.best_params_)

y_pred_GBC = GBC.predict(X_test)
GBC_test_acc = sum(np.where(y_pred_GBC == y_test, 1, 0))/y_test.shape[0]

print("Best Test Set Accuracy on tuned model : {:0.4f}".format(GBC_test_acc))
prfs = precision_recall_fscore_support(y_test, y_pred_GBC, average = 'macro')
print("Precision : ", prfs[0])
print("Recall : ", prfs[1])
print("F1 Score : ", prfs[2])

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV 1/2] END learning_rate=0.1, max_depth=5, n_estimators=5;, score=0.525 total time= 1.2min
[CV 2/2] END learning_rate=0.1, max_depth=5, n_estimators=5;, score=0.526 total time= 1.2min
[CV 1/2] END learning_rate=0.1, max_depth=5, n_estimators=10;, score=0.542 total time= 2.4min
[CV 2/2] END learning_rate=0.1, max_depth=5, n_estimators=10;, score=0.542 total time= 2.4min
[CV 1/2] END learning_rate=0.1, max_depth=10, n_estimators=5;, score=0.571 total time= 2.4min
[CV 2/2] END learning_rate=0.1, max_depth=10, n_estimators=5;, score=0.570 total time= 2.4min
[CV 1/2] END learning_rate=0.1, max_depth=10, n_estimators=10;, score=0.582 total time= 4.8min
[CV 2/2] END learning_rate=0.1, max_depth=10, n_estimators=10;, score=0.582 total time= 4.7min
[CV 1/2] END learning_rate=1, max_depth=5, n_estimators=5;, score=0.513 total time= 1.2min
[CV 2/2] END learning_rate=1, max_depth=5, n_estimators=5;, score=0.524 total time= 1.2min
[CV 1/

Hist Gradient Boosting Classifier

In [21]:
start_time = time.time()

parameters = {"max_depth": [5, 10],
            "learning_rate": [0.1, 1],
             'l2_regularization': np.logspace(-1, 2, 2)}

HGBC = GridSearchCV(HistGradientBoostingClassifier(), parameters, scoring = 'accuracy', verbose = 5, cv = 2)
HGBC.fit(X_dev, y_dev)

print("Time for model selection : {} seconds".format(time.time() - start_time))
print("Best Hyperparameters : ", HGBC.best_params_)

y_pred_HGBC = HGBC.predict(X_test)
HGBC_test_acc = sum(np.where(y_pred_HGBC == y_test, 1, 0))/y_test.shape[0]

print("Best Test Set Accuracy on tuned model : {:0.4f}".format(HGBC_test_acc))
prfs = precision_recall_fscore_support(y_test, y_pred_HGBC, average = 'macro')
print("Precision : ", prfs[0])
print("Recall : ", prfs[1])
print("F1 Score : ", prfs[2])

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV 1/2] END l2_regularization=0.1, learning_rate=0.1, max_depth=5;, score=0.583 total time=  25.3s
[CV 2/2] END l2_regularization=0.1, learning_rate=0.1, max_depth=5;, score=0.583 total time=  22.8s
[CV 1/2] END l2_regularization=0.1, learning_rate=0.1, max_depth=10;, score=0.591 total time=  25.4s
[CV 2/2] END l2_regularization=0.1, learning_rate=0.1, max_depth=10;, score=0.590 total time=  24.0s
[CV 1/2] END l2_regularization=0.1, learning_rate=1, max_depth=5;, score=0.326 total time=   2.9s
[CV 2/2] END l2_regularization=0.1, learning_rate=1, max_depth=5;, score=0.331 total time=   2.9s
[CV 1/2] END l2_regularization=0.1, learning_rate=1, max_depth=10;, score=0.470 total time=   2.9s
[CV 2/2] END l2_regularization=0.1, learning_rate=1, max_depth=10;, score=0.440 total time=   2.8s
[CV 1/2] END l2_regularization=100.0, learning_rate=0.1, max_depth=5;, score=0.575 total time=  24.1s
[CV 2/2] END l2_regularization=100.0, lear

XGBoost Classifier

In [31]:
from xgboost import XGBClassifier

start_time = time.time()

parameters = {'max_depth': [5, 10],
             'eta': np.logspace(-3, -1, 2),
             'lambda': np.logspace(-2, 2, 2)}

XGB = GridSearchCV(XGBClassifier(eval_metric = 'logloss'), parameters, scoring = 'accuracy', verbose = 5, cv = 2)
XGB.fit(X_dev, y_dev)

print("Time for model selection : {} seconds".format(time.time() - start_time))
print("Best Hyperparameters : ", XGB.best_params_)

y_pred_XGB = XGB.predict(X_test)
XGB_test_acc = sum(np.where(y_pred_XGB == y_test, 1, 0))/y_test.shape[0]

print("Best Test Set Accuracy on tuned model : {:0.4f}".format(XGB_test_acc))
prfs = precision_recall_fscore_support(y_test, y_pred_XGB, average = 'macro')
print("Precision : ", prfs[0])
print("Recall : ", prfs[1])
print("F1 Score : ", prfs[2])

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV 1/2] END eta=0.001, lambda=0.01, max_depth=5;, score=0.525 total time= 1.8min
[CV 2/2] END eta=0.001, lambda=0.01, max_depth=5;, score=0.525 total time= 1.8min
[CV 1/2] END eta=0.001, lambda=0.01, max_depth=10;, score=0.575 total time= 4.1min
[CV 2/2] END eta=0.001, lambda=0.01, max_depth=10;, score=0.572 total time= 4.0min
[CV 1/2] END eta=0.001, lambda=100.0, max_depth=5;, score=0.522 total time= 1.6min
[CV 2/2] END eta=0.001, lambda=100.0, max_depth=5;, score=0.522 total time= 1.7min
[CV 1/2] END eta=0.001, lambda=100.0, max_depth=10;, score=0.551 total time= 3.3min
[CV 2/2] END eta=0.001, lambda=100.0, max_depth=10;, score=0.549 total time= 3.4min
[CV 1/2] END .eta=0.1, lambda=0.01, max_depth=5;, score=0.573 total time= 1.9min
[CV 2/2] END .eta=0.1, lambda=0.01, max_depth=5;, score=0.572 total time= 1.8min
[CV 1/2] END eta=0.1, lambda=0.01, max_depth=10;, score=0.609 total time= 4.1min
[CV 2/2] END eta=0.1, lambda=0.01

AdaBoost Classifier

In [14]:
from sklearn.ensemble import AdaBoostClassifier

start_time = time.time()

parameters = {'n_estimators': [5, 10, 15, 20],
             'learning_rate': [0.1, 0.5, 0.75, 1, 1.25, 1.5]} 

#grid['n_estimators'] = [10, 50, 100, 500]
#grid['learning_rate'] = [0.0001, 0.001, 0.01, 0.1, 1.0]

ADB = GridSearchCV(AdaBoostClassifier(), parameters, scoring = 'accuracy', verbose = 5, cv = 2)
ADB.fit(X_dev, y_dev)

print("Time for model selection : {} seconds".format(time.time() - start_time))
print("Best Hyperparameters : ", ADB.best_params_)

y_pred_ADB = ADB.predict(X_test)
ADB_test_acc = sum(np.where(y_pred_ADB == y_test, 1, 0))/y_test.shape[0]

print("Best Test Set Accuracy on tuned model : {:0.4f}".format(ADB_test_acc))
prfs = precision_recall_fscore_support(y_test, y_pred_ADB, average = 'macro')
print("Precision : ", prfs[0])
print("Recall : ", prfs[1])
print("F1 Score : ", prfs[2])

Fitting 2 folds for each of 24 candidates, totalling 48 fits
[CV 1/2] END .learning_rate=0.1, n_estimators=5;, score=0.385 total time=   2.4s
[CV 2/2] END .learning_rate=0.1, n_estimators=5;, score=0.385 total time=   2.6s
[CV 1/2] END learning_rate=0.1, n_estimators=10;, score=0.385 total time=   5.1s
[CV 2/2] END learning_rate=0.1, n_estimators=10;, score=0.385 total time=   5.4s
[CV 1/2] END learning_rate=0.1, n_estimators=15;, score=0.409 total time=   7.5s
[CV 2/2] END learning_rate=0.1, n_estimators=15;, score=0.410 total time=   7.5s
[CV 1/2] END learning_rate=0.1, n_estimators=20;, score=0.411 total time=   9.8s
[CV 2/2] END learning_rate=0.1, n_estimators=20;, score=0.411 total time=  10.7s
[CV 1/2] END .learning_rate=0.5, n_estimators=5;, score=0.412 total time=   2.6s
[CV 2/2] END .learning_rate=0.5, n_estimators=5;, score=0.396 total time=   2.5s
[CV 1/2] END learning_rate=0.5, n_estimators=10;, score=0.435 total time=   5.1s
[CV 2/2] END learning_rate=0.5, n_estimators=10;

D:\Programs\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
data = pd.DataFrame({'Classifier':['Gradient Boosting', 'Hist Gradient Boosting', 'XGBoost','AdaBoost'],
        'Training Time':['72 min', '6.6 min', '53.1 min','5.4 min'],
        'Test Accuracy': [ 0.5857,0.5928,0.6149, 0.4550],
        'Test Pecision':[0.4555,0.4919,0.5503, 0.2319],
        'Test Recall':[0.3564,0.3641,0.3915, 0.1752]})
print(data)

               Classifier Training Time  Test Accuracy  Test Pecision  \
0       Gradient Boosting        72 min         0.5857         0.4555   
1  Hist Gradient Boosting       6.6 min         0.5928         0.4919   
2                 XGBoost      53.1 min         0.6149         0.5503   
3                AdaBoost       5.4 min         0.4550         0.2319   

   Test Recall  
0       0.3564  
1       0.3641  
2       0.3915  
3       0.1752  


In [29]:
# Take a look at the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred_ADB)

array([[ 3635,    26,     0, 16636,     1,    34,  3222,  4165,    52,
            0,     0,     0,     0],
       [  267,   145,     1,  2143,     3,    64,  5771,  1936,    36,
            0,     0,     0,     0],
       [  267,     8,     4,  2935,     2,    54,  1057,  1125,    13,
            0,     0,     0,     0],
       [ 1723,    57,     4, 26105,     4,    94,  2826,  3431,   157,
            0,     0,     0,     0],
       [  300,    22,     0,  4078,     5,    45,  3132,  2167,    62,
            0,     0,     0,     0],
       [   75,     2,     0,   457,     1,   140,   942,   434,     2,
            0,     0,     0,     0],
       [  554,    33,     0,  2163,     6,   173, 39062,  1915,    13,
            0,     0,     0,     0],
       [ 1233,    61,     0,  8281,     3,   118,  7688, 12034,    88,
            0,     0,     0,     0],
       [  979,     6,     0,  2330,     0,    34,  2142,  1426,    50,
            0,     0,     0,     0],
       [   48,     2,     0,